In [21]:
from pathlib import Path

import pandas as pd

from rdflib import Graph, URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS, OWL, Namespace

In [22]:
def parse_graph(path: Path, follow_imports: bool, depth: int) -> Graph:
    """
    Parses an RDF graph from the specified file path and optionally follows
    and parses imported RDF graphs up to a specified depth. The function
    produces a combined graph containing the original RDF content and any
    imported content up to the specified depth.

    :param path: Path to the RDF file to be parsed
    :type path: Path
    :param follow_imports: Indicates whether to follow imports and parse
        imported RDF documents
    :type follow_imports: bool
    :param depth: Depth up to which imported documents should be parsed.
        A depth of 0 indicates that only the given RDF file will be parsed,
        without following any imports
    :type depth: int
    :return: Parsed RDF graph with content from the specified file and
        optionally its imports up to the specified depth
    :rtype: Graph
    """
    g = Graph(); g.parse(path.as_posix())
    if follow_imports and depth > 0:
        from rdflib import URIRef
        frontier = [o for o in g.objects(None, OWL.imports) if isinstance(o, URIRef)]
        seen = set(); d = 0
        while frontier and d < depth:
            nxt = []
            for iri in frontier:
                if iri in seen: continue
                seen.add(iri)
                try:
                    g.parse(str(iri))
                    nxt += [o for o in g.objects(None, OWL.imports) if isinstance(o, URIRef)]
                except Exception:
                    pass
            frontier = nxt; d += 1
    return g


In [29]:
g = parse_graph(Path("../src/ccom.ttl"), True, 5)
print(g.serialize(format="turtle"))

@prefix cco: <https://www.commoncoreontologies.org/> .
@prefix dc1: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix obo: <http://purl.obolibrary.org/obo/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

obo:BFO_0000034 a owl:Class ;
    rdfs:label "function"@en ;
    dc11:identifier "064-BFO" ;
    rdfs:subClassOf obo:BFO_0000016 ;
    skos:definition "(Elucidation) A function is a disposition that exists in virtue of its bearer's physical make-up & this physical make-up is something the bearer possesses because it came into being either through evolution (in the case of natural biological entities) or through intentional design (in the case of artefacts) in order to re

In [35]:
from typing import Set
classes: Set[URIRef] = set()
ms_classes: Set[URIRef] = set()
# Find owl:Class
for s in g.subjects(RDF.type, OWL.Class):
    classes.add(s)

# Find rdfs:Class
for s in g.subjects(RDF.type, RDFS.Class):
    classes.add(s)

for s in classes:
    for label in g.objects(s, RDFS.label):
        if "Measurement Unit" in str(label):
            print(f"{s} - {label}")
            ms_classes.add(s)

https://www.commoncoreontologies.org/ont00000497 - Measurement Unit of Force
https://www.commoncoreontologies.org/ont00000969 - Measurement Unit of Speed
https://www.commoncoreontologies.org/ont00001345 - Measurement Unit of Impulse
https://www.commoncoreontologies.org/ont00000444 - Measurement Unit of Energy
https://www.commoncoreontologies.org/ont00001317 - Measurement Unit of Volume
https://www.commoncoreontologies.org/ont00001290 - Measurement Unit of Length
https://www.commoncoreontologies.org/ont00000074 - Measurement Unit of Acceleration
https://www.commoncoreontologies.org/ont00000852 - Measurement Unit of Work
https://www.commoncoreontologies.org/ont00000844 - Measurement Unit of Temperature
https://www.commoncoreontologies.org/ont00000229 - Measurement Unit of Power
https://www.commoncoreontologies.org/ont00000406 - Measurement Unit of Geocoordinate
https://www.commoncoreontologies.org/ont00001307 - Measurement Unit of Mass Flow Rate
https://www.commoncoreontologies.org/ont00

In [37]:
output = Graph()
prefix = "cco"
namespace = "https://www.commoncoreontologies.org/"
NS = Namespace(namespace)
output.bind(prefix, namespace)

for cls in ms_classes:
    cls.startswith("https://www.commoncoreontologies.org")
    output.add((cls, RDF.type, RDFS.Class))
    for label in g.objects(cls, RDFS.label):
        output.add((cls, RDFS.label, Literal(str(label))))


print(output.serialize(format="turtle"))


@prefix cco: <https://www.commoncoreontologies.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

cco:ont00000061 a rdfs:Class ;
    rdfs:label "Measurement Unit of Flow" .

cco:ont00000074 a rdfs:Class ;
    rdfs:label "Measurement Unit of Acceleration" .

cco:ont00000120 a rdfs:Class ;
    rdfs:label "Measurement Unit" .

cco:ont00000140 a rdfs:Class ;
    rdfs:label "Measurement Unit of Amount of Substance" .

cco:ont00000198 a rdfs:Class ;
    rdfs:label "Measurement Unit of Sound Level" .

cco:ont00000217 a rdfs:Class ;
    rdfs:label "Measurement Unit of Area" .

cco:ont00000229 a rdfs:Class ;
    rdfs:label "Measurement Unit of Power" .

cco:ont00000239 a rdfs:Class ;
    rdfs:label "Measurement Unit of Mass" .

cco:ont00000374 a rdfs:Class ;
    rdfs:label "Measurement Unit of Volumetric Flow Rate" .

cco:ont00000406 a rdfs:Class ;
    rdfs:label "Measurement Unit of Geocoordinate" .

cco:ont00000444 a rdfs:Class ;
    rdfs:label "Measurement Unit of Energy" .

cc